<a href="https://colab.research.google.com/github/jfcaro/SaturdaysAI-online-Learning-AI-with-TrikAI-Democratizando-la-formacion-/blob/main/Level_predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Level predictor training 

In [ ]:
# Install modules

! pip install spacy==3.1.4

In [31]:
# import modules

import pandas as pd
import spacy
import random
from spacy.util import minibatch
from spacy.training.example import Example

In [32]:
# gdrive connection

from google.colab import drive
drive.mount('/content/drive')
path = "/content/drive/My Drive/FormacionAI/Datasets/"
path1= "/content/drive/My Drive/FormacionAI/Datasets/new/"


! ls /content/drive/"My Drive/FormacionAI/Datasets"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
 2112.02086.pdf		        nlp
'AI Courses.gsheet'	        PlaylistsYouTube.xlsx
 Atention_is_all_you_need.pdf   ted_talks_en.csv
 Blog.gsheet		        VideosYouTube.xlsx
 Datasetsvideos7800.csv         WordClouds
 Difficulty_classification      youtube_transcrip.csv
 new			        youtube_transcrip_vector.csv


In [33]:
# loads the dataset with annotated levels 

df =pd.read_csv(path+"youtube_transcrip_vector.csv")[0:4430]

In [34]:
# this function provides train y vals datasets

def load_data(csv_file, split=0.95):
    data = pd.read_csv(csv_file)
    
    # Shuffle data
    train_data = data.sample(frac=1, random_state=7)
    
    texts = train_data.transcript.values
   
    labels = [{"BASIC": bool(y=="basic"), "ADVANCED": bool(y=="intermediate" or y =="advanced")} for y in train_data.level.values]
    split = int(len(train_data) * split)
    
    train_labels = [{"cats": labels} for labels in labels[:split]]
    val_labels = [{"cats": labels} for labels in labels[split:]]
    
    return texts[:split], train_labels, texts[split:], val_labels

train_texts, train_labels, val_texts, val_labels = load_data(path+"youtube_transcrip_vector.csv")

In [35]:
# some text and labels

print('Texts from training data\n------')
print(train_texts[:2])
print('\nLabels from training data\n------')
print(train_labels[:2])

Texts from training data
------
["And I would argue, it's about 8%. In fact, as we'll see, it will come out at 8 1/3%, mathematically. And the way to see this is this diagram is, this is the region in which the test as positive. By having a positive test, you know you're in this region, and nothing else matters. You know you're in this circle. But within this circle, the ratio of the cancerous region, relative to the entire region, is still pretty small. It increased. Obviously having a positive test, changes your cancer probability, but it only increases by a factor of about eight. As we'll see in a second. "
 "[Music] there are a lot of job advertisements on the internet even on reputed job advertising sites which never seem fake but after selection the so called recruiters start asking for money and bank details many of the candidates fall into their trap and lose a lot of money and the current job sometimes so it is better to identify whether a job advertisement posted on the site 

In [36]:
# Create an empty model

nlp = spacy.blank('en')

In [37]:
# verify spacy version

spacy.__version__

'3.1.4'

In [38]:
# Add the TextCategorizer to the empty model

textcat = nlp.add_pipe('textcat')

In [39]:
# Add labels to text classifier

textcat.add_label("BASIC")
textcat.add_label("ADVANCED")

1

In [40]:
# Training function

def train(model, train_data, optimizer, batch_size=8):
    losses = {}
    random.seed(1)
    random.shuffle(train_data)

    for batch in minibatch(train_data, size=batch_size):
        for text, labels in batch:
            doc = nlp.make_doc(text)
            example = Example.from_dict(doc, labels)
            # Update model with texts and labels
            model.update([example], sgd=optimizer, losses=losses)

    return losses

In [44]:
# Predictions 

def predict(nlp, texts): 
    # Use the model's tokenizer to tokenize each input text
    docs = [nlp.tokenizer(text) for text in texts]
    
    # Use textcat to get the scores for each doc
    textcat = nlp.get_pipe('textcat')
    scores = textcat.predict(docs)
    
    # From the scores, find the class with the highest score/probability
    predicted_class =  scores.argmax(axis=1)
    
    return predicted_class

In [45]:
# Accuracy function

def evaluate(model, texts, labels):

    # Get predictions from textcat model (using your predict method)
    predicted_class =  predict(model, texts)
    
    # From labels, get the true class as a list of integers (ADVANCED -> 1, BASIC -> 0)
    true_class = [int(each['cats']['ADVANCED']) for each in labels]
    
    # A boolean or int array indicating correct predictions
    correct_predictions = predicted_class == true_class
    
    # The accuracy, number of correct predictions divided by all predictions
    accuracy = correct_predictions.mean()
    
    return accuracy

In [47]:
# Fix seed for reproducibility

spacy.util.fix_random_seed(1)
random.seed(1)

# This may take a while to run!

optimizer = nlp.begin_training()
train_data = list(zip(train_texts, train_labels))

n_iters = 9

for i in range(n_iters):
    losses = train(nlp, train_data, optimizer)
    accuracy = evaluate(nlp, val_texts, val_labels)
    print(f"Loss: {losses['textcat']:.3f} \t Accuracy: {accuracy:.3f}")


Loss: 1202.267 	 Accuracy: 0.865
Loss: 763.276 	 Accuracy: 0.892
Loss: 532.591 	 Accuracy: 0.896
Loss: 384.904 	 Accuracy: 0.860
Loss: 268.863 	 Accuracy: 0.896
Loss: 221.850 	 Accuracy: 0.874
Loss: 163.763 	 Accuracy: 0.878
Loss: 138.731 	 Accuracy: 0.856
Loss: 127.494 	 Accuracy: 0.896


In [ ]:
# Shows some predictions

texts = val_texts[80:100]
predictions = predict(nlp, texts)

for p, t in zip(predictions, texts):
    print(f"{textcat.labels[p]}: {t} \n")

In [ ]:
# Save to disk 

nlp.to_disk(path +"nlp")